In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import pprint

# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

In [2]:
# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures))

all targeted structures: [u'VISal', u'VISl', u'VISp', u'VISpm']


In [3]:
# Download experiment containers for VISp experiments
visp_ecs = boc.get_experiment_containers(targeted_structures=['VISp'])
print("all VISp experiment containers: %d" % len(visp_ecs))

all VISp experiment containers: 25


In [4]:
# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print("all imaging depths: " + str(depths))

all imaging depths: [175, 275, 350, 375]


In [5]:
# Download a list of all stimuli
stims = boc.get_all_stimuli()
print("all stimuli:\n")
pprint.pprint(stims)

all stimuli:

['drifting_gratings',
 'locally_sparse_noise',
 'natural_movie_one',
 'natural_movie_three',
 'natural_movie_two',
 'natural_scenes',
 'spontaneous',
 'static_gratings']


In [6]:
# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

all cre lines: [u'Cux2-CreERT2', u'Rbp4-Cre', u'Rorb-IRES2-Cre', u'Scnn1a-Tg3-Cre']


In [7]:
import allensdk.brain_observatory.stimulus_info as stim_info

# Find the experiment with VISp, Cux2-CreERT2, 175, drifting gratings stimulus
crel = 'Cux2-CreERT2';
ts = 'VISp';
imdepth = 175;

exp = boc.get_ophys_experiments(cre_lines=[crel], 
                                stimuli=[stim_info.DRIFTING_GRATINGS],
                                targeted_structures=[ts],
                                imaging_depths=[imdepth])
print("Experiment with drifting gratings:")
pprint.pprint(exp)

Experiment with drifting gratings:
[{'age_days': 156.0,
  'cre_line': u'Cux2-CreERT2',
  'experiment_container_id': 511507650,
  'id': 502115959,
  'imaging_depth': 175,
  'session_type': u'three_session_A',
  'targeted_structure': u'VISp'},
 {'age_days': 124.0,
  'cre_line': u'Cux2-CreERT2',
  'experiment_container_id': 511510670,
  'id': 502205092,
  'imaging_depth': 175,
  'session_type': u'three_session_A',
  'targeted_structure': u'VISp'},
 {'age_days': 99.0,
  'cre_line': u'Cux2-CreERT2',
  'experiment_container_id': 511510718,
  'id': 510514474,
  'imaging_depth': 175,
  'session_type': u'three_session_A',
  'targeted_structure': u'VISp'},
 {'age_days': 111.0,
  'cre_line': u'Cux2-CreERT2',
  'experiment_container_id': 511510884,
  'id': 504637623,
  'imaging_depth': 175,
  'session_type': u'three_session_A',
  'targeted_structure': u'VISp'},
 {'age_days': 144.0,
  'cre_line': u'Cux2-CreERT2',
  'experiment_container_id': 511509529,
  'id': 501271265,
  'imaging_depth': 175,
  '

## get cell data for experiments

In [8]:
import numpy as np
from allensdk.brain_observatory.dff import compute_dff
import scipy
from scipy import io

In [ ]:
dpath = '/Users/Shuting/Documents/Yuste/projects/graph_ensemble/pa/data/'

#for m in range(len(exp)):
for m in [6,7]:
    
    # get cell data from the above experiments: first dataset
    print("Loading file ", m)
    data_set = boc.get_ophys_experiment_data(exp[m]['id'])

    print("Metadata from NWB file:")
    pprint.pprint(data_set.get_metadata())

    print("stimuli available in this file:")
    print(data_set.list_stimuli())
    
    # cell ids in experiment
    csid = data_set.get_cell_specimen_ids()
    for n in range(csid.size):
        #print('cell #', n, 'cell ID:', csid[n])
        # get trace
        time, raw_traces = data_set.get_fluorescence_traces(cell_specimen_ids=[csid[n]])
        # neuropil correction
        _, corrected_traces = data_set.get_corrected_fluorescence_traces(cell_specimen_ids=[csid[n]])
        # compute df/f
        dff_traces = compute_dff(np.array(corrected_traces))

        # store result
        if n==0:
            raw = np.zeros([len(data_set.get_cell_specimen_ids()),raw_traces.shape[1]])
            corrected = np.zeros([len(data_set.get_cell_specimen_ids()),corrected_traces.shape[1]])
            dff = np.zeros([len(data_set.get_cell_specimen_ids()),dff_traces.shape[1]])

        raw[n,:] = raw_traces
        corrected[n,:] = corrected_traces
        dff[n,:] = dff_traces
        
    # stimulus info
    stim_table = data_set.get_stimulus_table('drifting_gratings')
    stim_dict = {col_name: stim_table[col_name].values for col_name in stim_table.columns.values}
    
    # save results to mat format
    fname = dpath;
    fname += 'ecid_'
    fname += str(exp[m]['experiment_container_id'])
    result_dict = {'raw':raw, 'corrected':corrected, 'dff':dff, 'stim_info':stim_dict}
    
    scipy.io.savemat(fname, result_dict, appendmat=True)
    

In [ ]:
# from matplotlib import pyplot as plt
# %matplotlib inline

# # visualize result
# fig = plt.figure(figsize = [10.0,3.0])
# axes1 = fig.add_subplot(1, 3, 1)
# axes2 = fig.add_subplot(1, 3, 2)
# axes3 = fig.add_subplot(1, 3, 3)

# axes1.imshow(raw,aspect='auto')
# axes1.set_title('raw')
# axes2.imshow(corrected,aspect='auto')
# axes2.set_title('neuropil corrected')
# axes3.imshow(dff,aspect='auto')
# axes3.set_title('dF/F')

## import ROI mask

In [39]:
import numpy as np

dpath = '/Users/Shuting/Documents/Yuste/projects/graph_ensemble/pa/data/'
#for m in range(len(exp)):
for m in [6]:
    
    print("Loading file ", m)
    data_set = boc.get_ophys_experiment_data(exp[m]['id'])
    
    # get the specimen IDs for a few cells
    cids = data_set.get_cell_specimen_ids()#[:15:5]
    
    # get masks for specific cells
    roi_mask_list = data_set.get_roi_mask(cell_specimen_ids=cids)
    
    # save each roi mask
    for i in range(len(cids)):
        roi_mask = roi_mask_list[i]
        single_roi = roi_mask.get_mask_plane()
        if i==0:
            roi_all = np.zeros([len(cids),single_roi.shape[0],single_roi.shape[1]])
        roi_all[i,:,:] = single_roi
    
    # save results
    fname = dpath;
    fname += 'ecid_'
    fname += str(exp[m]['experiment_container_id'])
    fname += '_roi_mask'
    result_dict = {'roi_all':roi_all}
    scipy.io.savemat(fname, result_dict, appendmat=True)


('Loading file ', 6)


In [38]:
exp[6]['experiment_container_id']

511510650